In [ ]:
import pandas as pd
import plotly.graph_objects as go
import requests
import io
from datetime import datetime
from difflib import get_close_matches
import streamlit as st
import sys

# --- INICIALIZACIÓN DE PARÁMETROS ---
COLOR_POBTOT = "#0b0c10"
COLOR_HOMBRES = "#889064"
COLOR_MUJERES = "#ff9f18"
FONT_FAMILY = "Aptos Light"
municipio_query = None

# Parámetros inyectados desde Streamlit
if "active_palette" in locals():
    if len(active_palette) >= 1:
        COLOR_POBTOT = active_palette[0]
    if len(active_palette) >= 2:
        COLOR_HOMBRES = active_palette[1]
    if len(active_palette) >= 3:
        COLOR_MUJERES = active_palette[2]

if "active_font" in locals():
    FONT_FAMILY = active_font

if "MUNICIPIO_SELECCIONADO" in locals():
    municipio_query = MUNICIPIO_SELECCIONADO.strip()

if not municipio_query:
    st.error("❌ Error: No se ha seleccionado un municipio en la aplicación Streamlit.")
    sys.exit(1)

# ==============================
# CONFIGURACIÓN
# ==============================
URL = "https://www.datos.gob.mx/dataset/f2b9b220-3ef7-4e3a-bde6-87e1dac78c6a/resource/3c3092be-583e-4490-8c23-67ef9a64b198/download/pobproy_quinq1.csv"
ANOS_INICIO = datetime.now().year
ANOS_FIN = ANOS_INICIO + 5

# ==============================
# DESCARGA DEL DATASET
# ==============================
st.info("Descargando dataset de proyección poblacional desde datos.gob.mx...")

try:
    response = requests.get(URL, timeout=30)
    response.raise_for_status()

    # 🔥 Fix definitivo de acentos (mojibake)
    raw_bytes = response.content

    try:
        # Intento 1: decodificar como latin1
        text = raw_bytes.decode("latin1")
    except:
        # Intento 2: decodificar como utf-8 reemplazando errores
        text = raw_bytes.decode("utf-8", errors="replace")

    # Reparar mojibake (ej. RÃ­o → Río, MÃ©xico → México)
    text = text.encode("latin1", errors="ignore").decode("utf-8", errors="ignore")

    df = pd.read_csv(io.StringIO(text))
except Exception as e:
    st.error(f"❌ Error al descargar o leer el archivo: {e}")
    sys.exit(1)


df.columns = [col.strip() for col in df.columns]

# ==============================
# VALIDACIÓN Y BÚSQUEDA DE MUNICIPIOS
# ==============================
municipios_unicos = df["NOM_MUN"].astype(str).unique().tolist()
municipio_query_lower = municipio_query.lower()
coincidencias = []

# --- 1. Búsqueda de Subcadena (Devuelve todos los que contienen el texto) ---
coincidencias_subcadena = [
    m for m in municipios_unicos if municipio_query_lower in m.lower()
]

if coincidencias_subcadena:
    # Si encuentra subcadenas, simplemente las usa, pero ya NO muestra el mensaje
    coincidencias = coincidencias_subcadena

else:
    # --- 2. Búsqueda de Similitud (Fuzzy Matching) si la subcadena falla ---
    st.warning(f"⚠ No se encontró ningún municipio que contenga la cadena '{municipio_query}'. Buscando coincidencias aproximadas...")

    sugerencias_lower = get_close_matches(
        municipio_query_lower,
        [m.lower() for m in municipios_unicos],
        n=5,
        cutoff=0.6,
    )

    municipios_map = {m.lower(): m for m in municipios_unicos}
    coincidencias = [municipios_map[s] for s in sugerencias_lower]

    if coincidencias:
        st.warning(f"⚠ Usando aproximaciones: **{', '.join(coincidencias)}**")
    else:
        st.error(f"❌ No se encontraron coincidencias ni aproximaciones para: {municipio_query}")
        sys.exit(1)

# ==============================
# FILTRADO DE DATOS
# ==============================
st.subheader(f"Proyección Poblacional Quinquenal ({ANOS_INICIO} - {ANOS_FIN})")

coincidencias_lower = [m.lower() for m in coincidencias]

df_filtrado = df[
    (df["ANO"].between(ANOS_INICIO, ANOS_FIN))
    & (df["NOM_MUN"].astype(str).str.lower().isin(coincidencias_lower))
]

if df_filtrado.empty:
    st.error(
        f"❌ No hay datos de proyección entre {ANOS_INICIO}-{ANOS_FIN} "
        f"para los municipios detectados."
    )
    sys.exit(1)

# ==============================
# GENERACIÓN DE GRÁFICAS
# ==============================
se_generaron_graficas = False

for municipio in coincidencias:
    df_mun_all = df_filtrado[
        df_filtrado["NOM_MUN"].astype(str).str.lower() == municipio.lower()
    ]

    entidades = df_mun_all["NOM_ENT"].astype(str).unique().tolist()

    if not entidades:
        st.warning(f"⚠ No hay entidades asociadas a {municipio}. Se omite.")
        continue

    for entidad in entidades:
        df_ent = df_mun_all[df_mun_all["NOM_ENT"].astype(str) == entidad]

        df_agrupado = (
            df_ent.groupby(["ANO", "SEXO"])["POB_TOTAL"]
            .sum()
            .unstack(fill_value=0)
            .reset_index()
        )

        if "HOMBRES" not in df_agrupado.columns or "MUJERES" not in df_agrupado.columns:
            st.warning(
                f"⚠ Datos incompletos (sexo) para {municipio} - {entidad}. Se omite."
            )
            continue

        df_agrupado["TOTAL"] = df_agrupado["HOMBRES"] + df_agrupado["MUJERES"]

        anos = df_agrupado["ANO"].astype(str).tolist()
        poblacion_total = df_agrupado["TOTAL"].tolist()
        hombres = df_agrupado["HOMBRES"].tolist()
        mujeres = df_agrupado["MUJERES"].tolist()

        # ==============================
        # GRÁFICA
        # ==============================
        fig = go.Figure()

        fig.add_trace(go.Bar(
            x=anos,
            y=poblacion_total,
            name="Población Total",
            marker_color=COLOR_POBTOT,
            text=[f"{v:,}" for v in poblacion_total],
            textposition="outside",
        ))

        fig.add_trace(go.Bar(
            x=anos,
            y=hombres,
            name="Hombres",
            marker_color=COLOR_HOMBRES,
            text=[f"{v:,}" for v in hombres],
            textposition="outside",
        ))

        fig.add_trace(go.Bar(
            x=anos,
            y=mujeres,
            name="Mujeres",
            marker_color=COLOR_MUJERES,
            text=[f"{v:,}" for v in mujeres],
            textposition="outside",
        ))

        fig.update_layout(
            # === CAMBIO 1: Título Centrado ===
            title={
                "text": (
                    f"Proyección de población: {municipio}, {entidad}"
                    f"<br><span style='font-size:14px;'>({ANOS_INICIO}-{ANOS_FIN})</span>"
                ),
                "x": 0.5,
                "xanchor": "center",
            },
            barmode="group",
            yaxis=dict(showgrid=True, tickformat=",d"),
            plot_bgcolor="white",
            font=dict(family=FONT_FAMILY, size=14),
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=-0.2,
                xanchor="center",
                x=0.5,
            ),
            # === CAMBIO 2: Aumentar margen inferior para que quepa la fuente ===
            margin=dict(l=40, r=40, t=80, b=150)
        )

        # === CAMBIO 3: Agregar la fuente consultada en la parte inferior izquierda ===
        fig.add_annotation(
            text="Fuente: CONAPO (Datos Abiertos - Proyecciones de la Población)",
            xref="paper", yref="paper",
            x=0,      # Alineado a la izquierda
            y=-0.3,   # Coordenada negativa (debajo de la leyenda)
            showarrow=False,
            xanchor='left',
            yanchor='top',
            font=dict(size=12, color="gray", family=FONT_FAMILY)
        )

        st.markdown(f"#### {municipio}, {entidad}")
        st.plotly_chart(fig, use_container_width=True)
        se_generaron_graficas = True

# Mensaje final si no se generó ninguna gráfica
if not se_generaron_graficas:
    st.error(f"❌ No fue posible generar gráficas para el municipio: **{municipio_query}**.")